In [1]:
import glob
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import InceptionResNetV2

In [2]:
os.chdir('data')
test_paths = sorted(glob.glob('./test_imgs/*.jpg'))

In [4]:
base_model = InceptionResNetV2(
    input_shape=(270, 480, 3), include_top=False, weights='imagenet')

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512, activation='relu', input_dim=(7*13*1536))(x)
x = keras.layers.Dropout(0.1)(x)
predictions = keras.layers.Dense(48)(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='mean_squared_error',
              metrics=['accuracy'])

In [5]:
model.load_weights("model_checkpoint_callback_210318_12.h5")

In [6]:
X_test = []

for test_path in tqdm(test_paths):
    img = tf.io.read_file(test_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [270, 480])
    img = img/255
    X_test.append(img)

X_test = tf.stack(X_test, axis=0)
X_test.shape

100%|██████████| 1600/1600 [00:16<00:00, 99.73it/s] 


TensorShape([1600, 270, 480, 3])

In [8]:
pred = model.predict(X_test)

In [9]:
submission = pd.read_csv('./sample_submission.csv')
# image size를 1920x1080 -> 480x270으로 바꿔서 예측했으므로 * 4
submission.iloc[:, 1:] = pred*4

In [10]:
# submission
submission.to_csv('submission_210319_02.csv', index=False)